In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()


In [5]:
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()


In [6]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
final_df = orders.merge(users, on="user_id", how="left")
final_df = final_df.merge(restaurants, on="restaurant_id", how="left")

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [9]:
orders.columns
users.columns
restaurants.columns


Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [10]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [11]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [7]:
import pandas as pd
import sqlite3

# Load orders
orders = pd.read_csv("orders.csv")

# Load users
users = pd.read_json("users.json")

# Load restaurants from SQL
conn = sqlite3.connect("restaurants.db")
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# Merge datasets
final_df = orders.merge(users, on="user_id", how="left")
final_df = final_df.merge(restaurants, on="restaurant_id", how="left")


In [8]:
final_df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [9]:

list(final_df.columns)


['order_id',
 'user_id',
 'restaurant_id',
 'order_date',
 'total_amount',
 'restaurant_name_x',
 'name',
 'city',
 'membership',
 'restaurant_name_y',
 'cuisine',
 'rating']

In [11]:
# Convert date column to datetime
final_df["order_date"] = pd.to_datetime(final_df["order_date"], errors="coerce")

# Convert revenue to numeric
final_df["total_amount"] = pd.to_numeric(final_df["total_amount"], errors="coerce")


In [12]:
final_df = final_df.dropna(subset=["order_date", "total_amount"])


In [13]:
final_df = final_df.dropna(subset=["order_date", "total_amount"])


In [14]:
final_df["quarter"] = final_df["order_date"].dt.quarter


In [15]:
revenue_by_quarter = final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
print(revenue_by_quarter)


quarter
1    2041240.56
3    2009284.71
2    1986691.93
4    1974406.92
Name: total_amount, dtype: float64


In [16]:
revenue_by_quarter = final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
print(revenue_by_quarter)


quarter
1    2041240.56
3    2009284.71
2    1986691.93
4    1974406.92
Name: total_amount, dtype: float64


In [17]:
top_quarter = revenue_by_quarter.idxmax()
quarter_mapping = {1: "Q1 (Jan–Mar)", 2: "Q2 (Apr–Jun)", 3: "Q3 (Jul–Sep)", 4: "Q4 (Oct–Dec)"}
print("Quarter with highest revenue:", quarter_mapping[top_quarter])


Quarter with highest revenue: Q1 (Jan–Mar)


In [18]:
# Replace 'membership' with the correct column name if different
gold_orders_df = final_df[final_df["membership"] == "Gold"]


In [19]:
total_gold_orders = len(gold_orders_df)
print("Total orders by Gold members:", total_gold_orders)


Total orders by Gold members: 4987


In [20]:
hyderabad_orders = final_df[final_df["city"] == "Hyderabad"]


In [21]:
total_revenue_hyderabad = hyderabad_orders["total_amount"].sum()
total_revenue_hyderabad_rounded = round(total_revenue_hyderabad)
print("Total revenue from Hyderabad:", total_revenue_hyderabad_rounded)


Total revenue from Hyderabad: 1889367


In [22]:
distinct_users = final_df["user_id"].nunique()
print("Number of distinct users who placed at least one order:", distinct_users)


Number of distinct users who placed at least one order: 2883


In [23]:
gold_orders_df = final_df[final_df["membership"] == "Gold"]


In [24]:
avg_order_value = gold_orders_df["total_amount"].mean()
avg_order_value_rounded = round(avg_order_value, 2)
print("Average order value for Gold members:", avg_order_value_rounded)


Average order value for Gold members: 797.15


In [27]:
list(final_df.columns)


['order_id',
 'user_id',
 'restaurant_id',
 'order_date',
 'total_amount',
 'restaurant_name_x',
 'name',
 'city',
 'membership',
 'restaurant_name_y',
 'cuisine',
 'rating',
 'quarter']

In [28]:
final_df["rating"] = pd.to_numeric(final_df["rating"], errors="coerce")
